# GPU Training Hyperparmeter Optimization Analysis

**Purpose**: To provide an understanding of how various settings impact the speed of training a variety of models on a variety of datasets. As we scale up to larger models and larger datasets, knowing what settings are most efficient for training and the practical limits of the GPU are could save days of GPU time and facilitate more rapid iteration and experimentation.

**Author**: Christopher Risi

**Original Date**: 2025-10-05

**Modified Date**: 2025-10-05

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set plotting styles
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

Libraries imported successfully!


## 1. Batch Size Performance Analysis

We'll analyze the TTM batch size performance experiments to understand how different batch sizes affect training efficiency on the L40S GPU.

In [2]:
# Load and filter the model registry data
model_registry_path = "/u6/cjrisi/nocturnal/results/runs/ttm_finetune/model_registry.csv"
df = pd.read_csv(model_registry_path)

print(f"Total runs in registry: {len(df)}")

# Filter for batch size experiments
batch_size_experiments = df[df['experiment_name'] == 'ttm_batch_size_performance_limits_experiment'].copy()
print(f"Batch size experiments: {len(batch_size_experiments)}")

# Show completed experiments only
completed_experiments = batch_size_experiments[batch_size_experiments['status'] == 'completed'].copy()
print(f"Completed batch size experiments: {len(completed_experiments)}")

# Display the key columns
key_columns = ['run_id', 'job_id', 'batch_size', 'elapsed_time_seconds', 'final_train_loss', 
               'final_eval_loss', 'best_eval_loss', 'training_samples_per_second', 
               'peak_gpu_utilization', 'peak_memory_usage']

print("\nBatch Size Experiment Results:")
display_df = completed_experiments[key_columns].copy()
display_df = display_df.sort_values('batch_size')
display(display_df)

Total runs in registry: 41
Batch size experiments: 7
Completed batch size experiments: 7

Batch Size Experiment Results:


,run_id,job_id,batch_size,elapsed_time_seconds,final_train_loss,final_eval_loss,best_eval_loss,training_samples_per_second,peak_gpu_utilization,peak_memory_usage
25,run_20251005_185420_job1354922,1354922,64,865.373435,0.825482,0.821496,0.821496,1641.313,20.0,1.0
26,run_20251005_190854_job1354923,1354923,128,528.741835,0.825595,0.819627,0.819627,2728.473,38.0,25.0
27,run_20251005_191752_job1354924,1354924,256,367.629389,0.824284,0.818741,0.818741,4040.032,38.0,14.0
28,run_20251005_192407_job1354925,1354925,512,286.438970,0.822577,0.817276,0.817276,5300.703,42.0,29.0
29,run_20251005_192902_job1354926,1354926,1024,268.397645,0.821482,0.819813,0.819813,5739.498,53.0,46.0
30,run_20251005_193339_job1354927,1354927,2048,250.808962,0.822460,0.819894,0.819894,6169.580,72.0,73.0
31,run_20251005_193758_job1354928,1354928,4096,256.387371,0.826203,0.819134,0.819134,6024.558,98.0,98.0


In [3]:
# Create interactive scatter plots for batch size analysis
def create_batch_size_plots(df, metrics):
    """Create interactive scatter plots for different metrics vs batch size"""
    
    # Create subplots
    n_metrics = len(metrics)
    cols = 2
    rows = (n_metrics + 1) // 2
    
    fig = make_subplots(
        rows=rows, cols=cols,
        subplot_titles=[f"Batch Size vs {metric.replace('_', ' ').title()}" for metric in metrics],
        vertical_spacing=0.1,
        horizontal_spacing=0.1
    )
    
    colors = px.colors.qualitative.Set3
    
    for i, metric in enumerate(metrics):
        row = (i // cols) + 1
        col = (i % cols) + 1
        
        # Handle missing values
        plot_df = df[df[metric].notna()].copy()
        
        if len(plot_df) == 0:
            continue
            
        fig.add_trace(
            go.Scatter(
                x=plot_df['batch_size'],
                y=plot_df[metric],
                mode='markers+lines',
                name=metric.replace('_', ' ').title(),
                marker=dict(size=10, color=colors[i % len(colors)]),
                hovertemplate=f"<b>Batch Size:</b> %{{x}}<br>" +
                             f"<b>{metric.replace('_', ' ').title()}:</b> %{{y}}<br>" +
                             "<b>Job ID:</b> %{customdata}<extra></extra>",
                customdata=plot_df['job_id'],
                showlegend=False
            ),
            row=row, col=col
        )
        
        # Add trend line
        if len(plot_df) > 1:
            z = np.polyfit(plot_df['batch_size'], plot_df[metric], 1)
            p = np.poly1d(z)
            x_trend = np.linspace(plot_df['batch_size'].min(), plot_df['batch_size'].max(), 100)
            
            fig.add_trace(
                go.Scatter(
                    x=x_trend,
                    y=p(x_trend),
                    mode='lines',
                    name=f'{metric} trend',
                    line=dict(dash='dash', color=colors[i % len(colors)]),
                    showlegend=False,
                    hoverinfo='skip'
                ),
                row=row, col=col
            )
    
    fig.update_layout(
        height=300 * rows,
        title_text="TTM Batch Size Performance Analysis on L40S GPU",
        title_x=0.5,
        showlegend=True
    )
    
    return fig

# Define metrics to analyze
performance_metrics = [
    'elapsed_time_seconds',
    'training_samples_per_second', 
    'peak_gpu_utilization',
    'peak_memory_usage',
    'final_train_loss',
    'final_eval_loss'
]

# Create the plots
fig = create_batch_size_plots(completed_experiments, performance_metrics)
fig.show()

### 1.5. Learning Rate Performance Analysis

Let's analyze the new TTM learning rate performance experiments to understand how different learning rates affect training efficiency across different batch sizes on the L40S GPU.

In [4]:
# Load and filter the learning rate experiments
learning_rate_experiments = df[df['experiment_name'] == 'ttm_learning_rate_performance_limits_experiment'].copy()
print(f"Learning rate experiments: {len(learning_rate_experiments)}")

# Show completed experiments only
completed_lr_experiments = learning_rate_experiments[learning_rate_experiments['status'] == 'completed'].copy()
print(f"Completed learning rate experiments: {len(completed_lr_experiments)}")

# Display the key columns
lr_key_columns = ['run_id', 'job_id', 'batch_size', 'learning_rate', 'elapsed_time_seconds', 
                  'final_train_loss', 'final_eval_loss', 'best_eval_loss', 'training_samples_per_second', 
                  'peak_gpu_utilization', 'peak_memory_usage']

print("\nLearning Rate Experiment Results:")
display_df_lr = completed_lr_experiments[lr_key_columns].copy()
display_df_lr = display_df_lr.sort_values(['batch_size', 'learning_rate'])
display(display_df_lr)

Learning rate experiments: 9
Completed learning rate experiments: 9

Learning Rate Experiment Results:


,run_id,job_id,batch_size,learning_rate,elapsed_time_seconds,final_train_loss,final_eval_loss,best_eval_loss,training_samples_per_second,peak_gpu_utilization,peak_memory_usage
33,run_20251005_233627_job1354977,1354977,1024,0.001,507.770717,0.811814,0.824533,0.824533,5708.118,57.0,51.0
34,run_20251005_234503_job1354978,1354978,1024,0.005,509.715905,0.824712,0.821652,0.821652,5683.195,60.0,53.0
32,run_20251005_232734_job1354976,1354976,1024,0.010,524.280126,0.838426,0.827245,0.827245,5633.048,54.0,47.0
36,run_20251006_000202_job1354980,1354980,2048,0.001,491.244625,0.814586,0.820100,0.820100,5915.328,70.0,71.0
37,run_20251006_001022_job1354981,1354981,2048,0.005,489.882983,0.821168,0.820261,0.820261,5919.337,90.0,92.0
35,run_20251005_235341_job1354979,1354979,2048,0.010,493.005413,0.832115,0.819820,0.819820,5890.040,95.0,97.0
39,run_20251006_002654_job1354983,1354983,4096,0.001,485.470599,0.819694,0.818351,0.818351,5980.447,96.0,99.0
40,run_20251006_003508_job1354984,1354984,4096,0.005,486.809723,0.822396,0.819607,0.819607,5971.083,98.0,100.0
38,run_20251006_001840_job1354982,1354982,4096,0.010,486.073301,0.832251,0.821969,0.821969,5995.585,99.0,100.0


In [5]:
# Create interactive scatter plots for learning rate analysis with batch size as different lines
def create_learning_rate_plots(df, metrics):
    """Create interactive scatter plots for different metrics vs batch size, with learning rate as line colors"""
    
    # Create subplots
    n_metrics = len(metrics)
    cols = 2
    rows = (n_metrics + 1) // 2
    
    fig = make_subplots(
        rows=rows, cols=cols,
        subplot_titles=[f"Batch Size vs {metric.replace('_', ' ').title()}" for metric in metrics],
        vertical_spacing=0.1,
        horizontal_spacing=0.1
    )
    
    # Color palette for learning rates
    learning_rates = sorted(df['learning_rate'].unique())
    colors = px.colors.qualitative.Set2  # Different color palette for distinction
    
    for i, metric in enumerate(metrics):
        row = (i // cols) + 1
        col = (i % cols) + 1
        
        # Plot data for each learning rate
        for j, lr in enumerate(learning_rates):
            lr_data = df[df['learning_rate'] == lr].copy()
            
            # Handle missing values
            plot_df = lr_data[lr_data[metric].notna()].copy()
            
            if len(plot_df) == 0:
                continue
                
            # Sort by batch size for proper line connections
            plot_df = plot_df.sort_values('batch_size')
            
            fig.add_trace(
                go.Scatter(
                    x=plot_df['batch_size'],
                    y=plot_df[metric],
                    mode='markers+lines',
                    name=f'LR {lr}',
                    marker=dict(size=10, color=colors[j % len(colors)]),
                    line=dict(color=colors[j % len(colors)], width=2),
                    hovertemplate=f"<b>Learning Rate {lr}</b><br>" +
                                 f"<b>Batch Size:</b> %{{x}}<br>" +
                                 f"<b>{metric.replace('_', ' ').title()}:</b> %{{y}}<br>" +
                                 "<b>Job ID:</b> %{customdata}<extra></extra>",
                    customdata=plot_df['job_id'],
                    showlegend=True if i == 0 else False,  # Only show legend for first subplot
                    legendgroup=f'LR {lr}'
                ),
                row=row, col=col
            )
    
    fig.update_layout(
        height=300 * rows,
        title_text="TTM Learning Rate Performance Analysis - Batch Size vs Performance Metrics",
        title_x=0.5,
        showlegend=True
    )
    
    # Update axis labels
    for row_idx in range(1, rows + 1):
        for col_idx in range(1, cols + 1):
            fig.update_xaxes(title_text="Batch Size", row=row_idx, col=col_idx)
    
    return fig

# Define metrics to analyze for learning rate experiments
lr_performance_metrics = [
    'elapsed_time_seconds',
    'training_samples_per_second', 
    'peak_gpu_utilization',
    'peak_memory_usage',
    'final_train_loss',
    'final_eval_loss'
]

# Create the learning rate plots
if len(completed_lr_experiments) > 0:
    lr_fig = create_learning_rate_plots(completed_lr_experiments, lr_performance_metrics)
    lr_fig.show()
else:
    print("No completed learning rate experiments found for visualization")

In [6]:
# Analyze learning rate performance across batch sizes
def analyze_learning_rate_performance(df):
    """Analyze the learning rate performance data and provide insights"""
    
    # Clean data and remove any missing values
    clean_df = df.dropna(subset=['batch_size', 'learning_rate', 'elapsed_time_seconds', 'training_samples_per_second'])
    
    print("=== TTM Learning Rate Performance Analysis on L40S GPU ===\n")
    
    # 1. Performance by Learning Rate and Batch Size
    print("1. TRAINING PERFORMANCE BY LEARNING RATE AND BATCH SIZE:")
    print("-" * 60)
    
    # Create a pivot table for better visualization
    performance_summary = clean_df.pivot_table(
        index='batch_size',
        columns='learning_rate', 
        values=['training_samples_per_second', 'best_eval_loss', 'elapsed_time_seconds'],
        aggfunc='mean'
    ).round(2)
    
    print("Training Samples per Second:")
    print(performance_summary['training_samples_per_second'])
    print("\nBest Evaluation Loss:")
    print(performance_summary['best_eval_loss'])
    print("\nElapsed Time (seconds):")  
    print(performance_summary['elapsed_time_seconds'])
    
    print("\n" + "="*60 + "\n")
    
    # 2. Learning Rate Impact Analysis
    print("2. LEARNING RATE IMPACT ANALYSIS:")
    print("-" * 40)
    
    lr_analysis = clean_df.groupby('learning_rate').agg({
        'training_samples_per_second': ['mean', 'std'],
        'best_eval_loss': ['mean', 'std'],
        'final_train_loss': ['mean', 'std'],
        'elapsed_time_seconds': ['mean', 'std']
    }).round(4)
    
    print("Average Performance by Learning Rate:")
    for lr in sorted(clean_df['learning_rate'].unique()):
        lr_data = clean_df[clean_df['learning_rate'] == lr]
        avg_throughput = lr_data['training_samples_per_second'].mean()
        avg_loss = lr_data['best_eval_loss'].mean()
        avg_time = lr_data['elapsed_time_seconds'].mean()
        
        print(f"   LR {lr:5.3f}: {avg_throughput:6.1f} samples/sec, {avg_loss:.4f} eval loss, {avg_time:6.1f}s")
    
    # Find optimal learning rates
    best_throughput_lr = clean_df.loc[clean_df['training_samples_per_second'].idxmax(), 'learning_rate']
    best_loss_lr = clean_df.loc[clean_df['best_eval_loss'].idxmin(), 'learning_rate']
    
    print(f"\n🚀 Best throughput: LR {best_throughput_lr}")
    print(f"🎯 Best model performance: LR {best_loss_lr}")
    
    print("\n" + "="*60 + "\n")
    
    # 3. Batch Size vs Learning Rate Interaction
    print("3. BATCH SIZE vs LEARNING RATE INTERACTION:")
    print("-" * 45)
    
    print("Performance Matrix (Throughput samples/sec):")
    throughput_matrix = clean_df.pivot_table(
        index='batch_size', 
        columns='learning_rate', 
        values='training_samples_per_second',
        aggfunc='mean'
    ).round(1)
    print(throughput_matrix)
    
    print("\nModel Quality Matrix (Best Eval Loss):")
    loss_matrix = clean_df.pivot_table(
        index='batch_size', 
        columns='learning_rate', 
        values='best_eval_loss',
        aggfunc='mean'
    ).round(4)
    print(loss_matrix)
    
    # Find best combinations
    best_combo_idx = clean_df.loc[clean_df['best_eval_loss'].idxmin()]
    fastest_combo_idx = clean_df.loc[clean_df['training_samples_per_second'].idxmax()]
    
    print(f"\n💡 Best overall combination:")
    print(f"   Batch Size: {best_combo_idx['batch_size']}, LR: {best_combo_idx['learning_rate']}")
    print(f"   Best eval loss: {best_combo_idx['best_eval_loss']:.4f}")
    print(f"   Throughput: {best_combo_idx['training_samples_per_second']:.1f} samples/sec")
    
    print(f"\n⚡ Fastest training combination:")
    print(f"   Batch Size: {fastest_combo_idx['batch_size']}, LR: {fastest_combo_idx['learning_rate']}")
    print(f"   Throughput: {fastest_combo_idx['training_samples_per_second']:.1f} samples/sec")
    print(f"   Best eval loss: {fastest_combo_idx['best_eval_loss']:.4f}")
    
    return clean_df

# Perform the learning rate analysis
if len(completed_lr_experiments) > 0:
    lr_analysis_df = analyze_learning_rate_performance(completed_lr_experiments)
else:
    print("No completed learning rate experiments found for analysis")

=== TTM Learning Rate Performance Analysis on L40S GPU ===

1. TRAINING PERFORMANCE BY LEARNING RATE AND BATCH SIZE:
------------------------------------------------------------
Training Samples per Second:
learning_rate    0.001    0.005    0.010
batch_size                              
1024           5708.12  5683.20  5633.05
2048           5915.33  5919.34  5890.04
4096           5980.45  5971.08  5995.58

Best Evaluation Loss:
learning_rate  0.001  0.005  0.010
batch_size                        
1024            0.82   0.82   0.83
2048            0.82   0.82   0.82
4096            0.82   0.82   0.82

Elapsed Time (seconds):
learning_rate   0.001   0.005   0.010
batch_size                           
1024           507.77  509.72  524.28
2048           491.24  489.88  493.01
4096           485.47  486.81  486.07


2. LEARNING RATE IMPACT ANALYSIS:
----------------------------------------
Average Performance by Learning Rate:
   LR 0.001: 5868.0 samples/sec, 0.8210 eval loss,  494.8s
 

In [7]:
# Create a comprehensive comparison visualization
def create_comprehensive_comparison_plots(batch_df, lr_df):
    """Create side-by-side comparison of batch size and learning rate effects"""
    
    fig = make_subplots(
        rows=2, cols=3,
        subplot_titles=[
            "Batch Size vs Throughput (Batch Size Exp)",
            "Batch Size vs Throughput (Learning Rate Exp)", 
            "Learning Rate Effect on Throughput",
            "Batch Size vs Model Quality (Best Eval Loss)",
            "Learning Rate vs Model Quality", 
            "Training Efficiency Comparison"
        ],
        vertical_spacing=0.15,
        horizontal_spacing=0.1
    )
    
    colors_batch = px.colors.qualitative.Set1
    colors_lr = px.colors.qualitative.Set2
    
    # 1. Batch Size vs Throughput (from batch size experiments)
    batch_clean = batch_df.dropna(subset=['batch_size', 'training_samples_per_second'])
    if len(batch_clean) > 0:
        fig.add_trace(
            go.Scatter(
                x=batch_clean['batch_size'],
                y=batch_clean['training_samples_per_second'],
                mode='markers+lines',
                name='Batch Size Exp',
                marker=dict(size=10, color=colors_batch[0]),
                line=dict(color=colors_batch[0], width=3),
                hovertemplate="<b>Batch Size Exp</b><br>Batch Size: %{x}<br>Throughput: %{y:.0f}<extra></extra>",
                showlegend=True
            ),
            row=1, col=1
        )
    
    # 2. Batch Size vs Throughput (from learning rate experiments)
    lr_clean = lr_df.dropna(subset=['batch_size', 'training_samples_per_second', 'learning_rate'])
    learning_rates = sorted(lr_clean['learning_rate'].unique())
    
    for i, lr in enumerate(learning_rates):
        lr_data_subset = lr_clean[lr_clean['learning_rate'] == lr].sort_values('batch_size')
        fig.add_trace(
            go.Scatter(
                x=lr_data_subset['batch_size'],
                y=lr_data_subset['training_samples_per_second'],
                mode='markers+lines',
                name=f'LR {lr}',
                marker=dict(size=8, color=colors_lr[i]),
                line=dict(color=colors_lr[i], width=2),
                hovertemplate=f"<b>LR {lr}</b><br>Batch Size: %{{x}}<br>Throughput: %{{y:.0f}}<extra></extra>",
                showlegend=True,
                legendgroup=f'lr_{lr}'
            ),
            row=1, col=2
        )
    
    # 3. Learning Rate Effect on Throughput (aggregated across batch sizes)
    lr_agg = lr_clean.groupby('learning_rate')['training_samples_per_second'].mean().reset_index()
    fig.add_trace(
        go.Scatter(
            x=lr_agg['learning_rate'],
            y=lr_agg['training_samples_per_second'],
            mode='markers+lines',
            name='Avg Throughput',
            marker=dict(size=12, color=colors_lr[3]),
            line=dict(color=colors_lr[3], width=3),
            hovertemplate="<b>Average</b><br>Learning Rate: %{x}<br>Throughput: %{y:.0f}<extra></extra>",
            showlegend=False
        ),
        row=1, col=3
    )
    
    # 4. Batch Size vs Model Quality (Best Eval Loss) from batch size experiments
    if len(batch_clean) > 0 and 'best_eval_loss' in batch_clean.columns:
        batch_loss_clean = batch_clean.dropna(subset=['best_eval_loss'])
        fig.add_trace(
            go.Scatter(
                x=batch_loss_clean['batch_size'],
                y=batch_loss_clean['best_eval_loss'],
                mode='markers+lines',
                name='Batch Size Exp Loss',
                marker=dict(size=10, color=colors_batch[1]),
                line=dict(color=colors_batch[1], width=3),
                hovertemplate="<b>Batch Size Exp</b><br>Batch Size: %{x}<br>Best Loss: %{y:.4f}<extra></extra>",
                showlegend=False
            ),
            row=2, col=1
        )
    
    # 5. Learning Rate vs Model Quality
    lr_loss_agg = lr_clean.groupby('learning_rate')['best_eval_loss'].mean().reset_index()
    fig.add_trace(
        go.Scatter(
            x=lr_loss_agg['learning_rate'],
            y=lr_loss_agg['best_eval_loss'],
            mode='markers+lines',
            name='Avg Loss',
            marker=dict(size=12, color=colors_lr[4]),
            line=dict(color=colors_lr[4], width=3),
            hovertemplate="<b>Average</b><br>Learning Rate: %{x}<br>Best Loss: %{y:.4f}<extra></extra>",
            showlegend=False
        ),
        row=2, col=2
    )
    
    # 6. Training Efficiency Comparison (samples per second per minute of training)
    if len(batch_clean) > 0:
        batch_efficiency = batch_clean['training_samples_per_second'] / (batch_clean['elapsed_time_seconds'] / 60)
        fig.add_trace(
            go.Scatter(
                x=batch_clean['batch_size'],
                y=batch_efficiency,
                mode='markers',
                name='Batch Exp Efficiency',
                marker=dict(size=10, color=colors_batch[2], symbol='circle'),
                hovertemplate="<b>Batch Size Exp</b><br>Batch Size: %{x}<br>Efficiency: %{y:.1f}<extra></extra>",
                showlegend=False
            ),
            row=2, col=3
        )
    
    # Add learning rate experiment efficiency
    for i, lr in enumerate(learning_rates):
        lr_subset = lr_clean[lr_clean['learning_rate'] == lr]
        lr_efficiency = lr_subset['training_samples_per_second'] / (lr_subset['elapsed_time_seconds'] / 60)
        fig.add_trace(
            go.Scatter(
                x=lr_subset['batch_size'],
                y=lr_efficiency,
                mode='markers',
                name=f'LR {lr} Efficiency',
                marker=dict(size=8, color=colors_lr[i], symbol='diamond'),
                hovertemplate=f"<b>LR {lr}</b><br>Batch Size: %{{x}}<br>Efficiency: %{{y:.1f}}<extra></extra>",
                showlegend=False,
                legendgroup=f'lr_{lr}'
            ),
            row=2, col=3
        )
    
    # Update layout
    fig.update_layout(
        height=800,
        width=1600,
        title_text="Comprehensive TTM Performance Analysis: Batch Size vs Learning Rate Effects",
        title_x=0.5,
        showlegend=True
    )
    
    # Update axis labels
    fig.update_xaxes(title_text="Batch Size", row=1, col=1)
    fig.update_xaxes(title_text="Batch Size", row=1, col=2)
    fig.update_xaxes(title_text="Learning Rate", row=1, col=3)
    fig.update_xaxes(title_text="Batch Size", row=2, col=1)
    fig.update_xaxes(title_text="Learning Rate", row=2, col=2)
    fig.update_xaxes(title_text="Batch Size", row=2, col=3)
    
    fig.update_yaxes(title_text="Samples/sec", row=1, col=1)
    fig.update_yaxes(title_text="Samples/sec", row=1, col=2)
    fig.update_yaxes(title_text="Samples/sec", row=1, col=3)
    fig.update_yaxes(title_text="Best Eval Loss", row=2, col=1)
    fig.update_yaxes(title_text="Best Eval Loss", row=2, col=2)
    fig.update_yaxes(title_text="Efficiency", row=2, col=3)
    
    return fig

# Create comprehensive comparison
if len(completed_experiments) > 0 and len(completed_lr_experiments) > 0:
    comp_fig = create_comprehensive_comparison_plots(completed_experiments, completed_lr_experiments)
    comp_fig.show()
    
    print("\n" + "="*70)
    print("🔍 KEY INSIGHTS FROM COMPREHENSIVE COMPARISON:")
    print("="*70)
    
    # Quick comparison insights
    batch_exp_max_throughput = completed_experiments['training_samples_per_second'].max()
    lr_exp_max_throughput = completed_lr_experiments['training_samples_per_second'].max()
    
    batch_exp_best_loss = completed_experiments['best_eval_loss'].min()
    lr_exp_best_loss = completed_lr_experiments['best_eval_loss'].min()
    
    print(f"📊 Maximum Throughput Comparison:")
    print(f"   Batch Size Experiments: {batch_exp_max_throughput:.0f} samples/sec")
    print(f"   Learning Rate Experiments: {lr_exp_max_throughput:.0f} samples/sec")
    print(f"   Difference: {lr_exp_max_throughput - batch_exp_max_throughput:.0f} samples/sec")
    
    print(f"\n🎯 Best Model Quality Comparison:")
    print(f"   Batch Size Experiments: {batch_exp_best_loss:.4f} eval loss")
    print(f"   Learning Rate Experiments: {lr_exp_best_loss:.4f} eval loss")
    print(f"   Improvement: {batch_exp_best_loss - lr_exp_best_loss:.4f} loss reduction")
    
    if lr_exp_best_loss < batch_exp_best_loss:
        print("   ✅ Learning rate optimization improved model quality!")
    else:
        print("   ⚠️ Learning rate experiments did not improve model quality")
        
else:
    print("Insufficient data for comprehensive comparison")


🔍 KEY INSIGHTS FROM COMPREHENSIVE COMPARISON:
📊 Maximum Throughput Comparison:
   Batch Size Experiments: 6170 samples/sec
   Learning Rate Experiments: 5996 samples/sec
   Difference: -174 samples/sec

🎯 Best Model Quality Comparison:
   Batch Size Experiments: 0.8173 eval loss
   Learning Rate Experiments: 0.8184 eval loss
   Improvement: -0.0011 loss reduction
   ⚠️ Learning rate experiments did not improve model quality


## 2. Training Loss Analysis

Now let's examine the training progress over time by analyzing the trainer_state.json files from the latest batch size experiments.

In [8]:
# Function to load training history from trainer_state.json files
def load_training_history(base_path):
    """Load training history from trainer_state.json files in the latest batch size experiments"""
    
    training_data = []
    
    # Get the 6 latest training directories (all from Oct 5th batch size experiments)
    latest_dirs = [
        "2025-10-05_19-38-13",  # 4096 batch size
        "2025-10-05_19-33-54",  # 2048 batch size  
        "2025-10-05_19-29-19",  # 1024 batch size
        "2025-10-05_19-24-23",  # 512 batch size
        "2025-10-05_19-18-07",  # 256 batch size
        "2025-10-05_19-09-10",  # 128 batch size
        "2025-10-05_18-54-40"   # 64 batch size
    ]
    
    # Map directories to batch sizes based on the model registry
    batch_size_map = {
        "2025-10-05_19-38-13": 4096,
        "2025-10-05_19-33-54": 2048,
        "2025-10-05_19-29-19": 1024,
        "2025-10-05_19-24-23": 512,
        "2025-10-05_19-18-07": 256,
        "2025-10-05_19-09-10": 128,
        "2025-10-05_18-54-40": 64,
    }
    
    for dir_name in latest_dirs:
        dir_path = Path(base_path) / dir_name / "5min_patients" / "output"
        
        if not dir_path.exists():
            print(f"Directory not found: {dir_path}")
            continue
            
        # Find the latest checkpoint directory
        checkpoint_dirs = [d for d in dir_path.iterdir() if d.is_dir() and d.name.startswith("checkpoint-")]
        if not checkpoint_dirs:
            print(f"No checkpoint directories found in {dir_path}")
            continue
            
        checkpoint_dirs.sort(key=lambda x: int(x.name.split("-")[1]), reverse=True)
        latest_checkpoint = checkpoint_dirs[0]
        
        trainer_state_file = latest_checkpoint / "trainer_state.json"
        
        if trainer_state_file.exists():
            print(f"Loading training history from: {trainer_state_file}")
            
            with open(trainer_state_file, 'r') as f:
                trainer_state = json.load(f)
                
            log_history = trainer_state.get('log_history', [])
            batch_size = batch_size_map.get(dir_name, 'unknown')
            
            for entry in log_history:
                entry_data = entry.copy()
                entry_data['batch_size'] = batch_size
                entry_data['experiment_dir'] = dir_name
                training_data.append(entry_data)
        else:
            print(f"trainer_state.json not found in {latest_checkpoint}")
    
    return pd.DataFrame(training_data)

# Load the training history data
models_path = "/u6/cjrisi/nocturnal/models/ttm/kaggle_brisT1D"
training_df = load_training_history(models_path)

print(f"Loaded training data: {len(training_df)} records")
print(f"Unique batch sizes: {sorted(training_df['batch_size'].unique())}")
print(f"Columns: {list(training_df.columns)}")

# Display sample data
training_df.head(20)

Loading training history from: /u6/cjrisi/nocturnal/models/ttm/kaggle_brisT1D/2025-10-05_19-38-13/5min_patients/output/checkpoint-340/trainer_state.json
Loading training history from: /u6/cjrisi/nocturnal/models/ttm/kaggle_brisT1D/2025-10-05_19-33-54/5min_patients/output/checkpoint-670/trainer_state.json
Loading training history from: /u6/cjrisi/nocturnal/models/ttm/kaggle_brisT1D/2025-10-05_19-29-19/5min_patients/output/checkpoint-1340/trainer_state.json
Loading training history from: /u6/cjrisi/nocturnal/models/ttm/kaggle_brisT1D/2025-10-05_19-24-23/5min_patients/output/checkpoint-2670/trainer_state.json
Loading training history from: /u6/cjrisi/nocturnal/models/ttm/kaggle_brisT1D/2025-10-05_19-18-07/5min_patients/output/checkpoint-5330/trainer_state.json
Loading training history from: /u6/cjrisi/nocturnal/models/ttm/kaggle_brisT1D/2025-10-05_19-09-10/5min_patients/output/checkpoint-10660/trainer_state.json
Loading training history from: /u6/cjrisi/nocturnal/models/ttm/kaggle_brisT1D

,epoch,grad_norm,learning_rate,loss,step,batch_size,experiment_dir,eval_custom_error,eval_loss,eval_runtime,eval_samples_per_second,eval_steps_per_second
0,0.029412,0.664836,0.002000,0.9910,1,4096,2025-10-05_19-38-13,NaN,NaN,NaN,NaN,NaN
1,2.941176,0.054153,0.001980,0.8414,100,4096,2025-10-05_19-38-13,NaN,NaN,NaN,NaN,NaN
2,5.882353,0.078760,0.001961,0.8250,200,4096,2025-10-05_19-38-13,NaN,NaN,NaN,NaN,NaN
3,8.823529,0.090141,0.001941,0.8162,300,4096,2025-10-05_19-38-13,NaN,NaN,NaN,NaN,NaN
4,0.014925,0.648445,0.002000,0.9891,1,2048,2025-10-05_19-33-54,NaN,NaN,NaN,NaN,NaN
5,1.492537,0.080005,0.001980,0.8435,100,2048,2025-10-05_19-33-54,NaN,NaN,NaN,NaN,NaN
6,2.985075,0.106652,0.001961,0.8281,200,2048,2025-10-05_19-33-54,NaN,NaN,NaN,NaN,NaN
7,4.477612,0.121914,0.001941,0.8229,300,2048,2025-10-05_19-33-54,NaN,NaN,NaN,NaN,NaN
8,5.970149,0.185538,0.001922,0.8192,400,2048,2025-10-05_19-33-54,NaN,NaN,NaN,NaN,NaN
9,7.462687,0.115421,0.001903,0.8127,500,2048,2025-10-05_19-33-54,NaN,NaN,NaN,NaN,NaN


In [9]:
# Create training loss visualization
from turtle import width


def create_training_plots(df):
    """Create training loss and learning rate plots by batch size"""
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            "Training Loss vs Epoch", 
            "Training Loss vs Step",
            "Learning Rate vs Epoch",
            "Learning Rate vs Step"
        ],
        vertical_spacing=0.1,
        horizontal_spacing=0.1
    )
    
    # Color palette
    colors = px.colors.qualitative.Set1
    batch_sizes = sorted(df['batch_size'].unique())
    
    for i, batch_size in enumerate(batch_sizes):
        if batch_size == 'unknown':
            continue
            
        batch_data = df[df['batch_size'] == batch_size].copy()
        batch_data = batch_data.sort_values('step')
        
        color = colors[i % len(colors)]
        name = f"Batch Size {batch_size}"
        
        # Training Loss vs Epoch
        fig.add_trace(
            go.Scatter(
                x=batch_data['epoch'],
                y=batch_data['loss'],
                mode='lines+markers',
                name=name,
                line=dict(color=color),
                marker=dict(size=4),
                showlegend=True,
                legendgroup=name,
                hovertemplate=f"<b>{name}</b><br>" +
                             "Epoch: %{x:.2f}<br>" +
                             "Loss: %{y:.4f}<extra></extra>"
            ),
            row=1, col=1
        )
        
        # Training Loss vs Step
        fig.add_trace(
            go.Scatter(
                x=batch_data['step'],
                y=batch_data['loss'],
                mode='lines+markers',
                name=name,
                line=dict(color=color),
                marker=dict(size=4),
                showlegend=False,
                legendgroup=name,
                hovertemplate=f"<b>{name}</b><br>" +
                             "Step: %{x}<br>" +
                             "Loss: %{y:.4f}<extra></extra>"
            ),
            row=1, col=2
        )
        
        # Learning Rate vs Epoch
        if 'learning_rate' in batch_data.columns:
            fig.add_trace(
                go.Scatter(
                    x=batch_data['epoch'],
                    y=batch_data['learning_rate'],
                    mode='lines+markers',
                    name=name,
                    line=dict(color=color),
                    marker=dict(size=4),
                    showlegend=False,
                    legendgroup=name,
                    hovertemplate=f"<b>{name}</b><br>" +
                                 "Epoch: %{x:.2f}<br>" +
                                 "Learning Rate: %{y:.6f}<extra></extra>"
                ),
                row=2, col=1
            )
            
            # Learning Rate vs Step
            fig.add_trace(
                go.Scatter(
                    x=batch_data['step'],
                    y=batch_data['learning_rate'],
                    mode='lines+markers',
                    name=name,
                    line=dict(color=color),
                    marker=dict(size=4),
                    showlegend=False,
                    legendgroup=name,
                    hovertemplate=f"<b>{name}</b><br>" +
                                 "Step: %{x}<br>" +
                                 "Learning Rate: %{y:.6f}<extra></extra>"
                ),
                row=2, col=2
            )
    
    # Update layout
    fig.update_layout(
        height=800,
        width=2000,
        title_text="TTM Training Progress Analysis - Batch Size Comparison",
        title_x=0.5,
        showlegend=True
    )
    
    # Update axis labels
    fig.update_xaxes(title_text="Epoch", row=1, col=1)
    fig.update_xaxes(title_text="Step", row=1, col=2)
    fig.update_xaxes(title_text="Epoch", row=2, col=1)
    fig.update_xaxes(title_text="Step", row=2, col=2)
    
    fig.update_yaxes(title_text="Loss", row=1, col=1)
    fig.update_yaxes(title_text="Loss", row=1, col=2)
    fig.update_yaxes(title_text="Learning Rate", row=2, col=1)
    fig.update_yaxes(title_text="Learning Rate", row=2, col=2)
    
    return fig

# Create and display the training plots
if len(training_df) > 0:
    training_fig = create_training_plots(training_df)
    training_fig.show()
else:
    print("No training data available for visualization")

## 3. Performance Analysis Summary

Let's analyze the key findings from our batch size performance experiments.

In [10]:
# Create comprehensive analysis summary
def analyze_batch_size_performance(df):
    """Analyze the batch size performance data and provide insights"""
    
    # Clean data and remove any missing values
    clean_df = df.dropna(subset=['batch_size', 'elapsed_time_seconds', 'training_samples_per_second'])
    clean_df = clean_df[clean_df['batch_size'] != 'unknown']
    
    print("=== TTM Batch Size Performance Analysis on L40S GPU ===\n")
    
    # 1. Training Speed Analysis
    print("1. TRAINING THROUGHPUT ANALYSIS:")
    print("-" * 40)
    speed_summary = clean_df.groupby('batch_size').agg({
        'training_samples_per_second': ['mean', 'std'],
        'elapsed_time_seconds': ['mean', 'std'],
        'peak_gpu_utilization': ['mean', 'std'],
        'peak_memory_usage': ['mean', 'std']
    }).round(2)
    
    # Flatten column names
    speed_summary.columns = ['_'.join(col).strip() for col in speed_summary.columns]
    print(speed_summary)
    
    # Find optimal batch size for throughput
    max_throughput_idx = clean_df.loc[clean_df['training_samples_per_second'].idxmax()]
    print(f"\n🚀 Highest throughput: {max_throughput_idx['training_samples_per_second']:.1f} samples/sec")
    print(f"   Achieved with batch size: {max_throughput_idx['batch_size']}")
    print(f"   GPU utilization: {max_throughput_idx['peak_gpu_utilization']:.1f}%")
    print(f"   Memory usage: {max_throughput_idx['peak_memory_usage']:.1f} GB")
    
    print("\n" + "="*60 + "\n")
    
    # 2. GPU Resource Utilization
    print("2. GPU RESOURCE UTILIZATION ANALYSIS:")
    print("-" * 45)
    
    # Memory usage efficiency
    clean_df['memory_efficiency'] = clean_df['training_samples_per_second'] / clean_df['peak_memory_usage']
    memory_efficient_idx = clean_df.loc[clean_df['memory_efficiency'].idxmax()]
    
    print(f"📊 Memory Usage Patterns:")
    for batch_size in sorted(clean_df['batch_size'].unique()):
        batch_data = clean_df[clean_df['batch_size'] == batch_size]
        avg_memory = batch_data['peak_memory_usage'].mean()
        avg_util = batch_data['peak_gpu_utilization'].mean()
        avg_throughput = batch_data['training_samples_per_second'].mean()
        print(f"   Batch {batch_size:4d}: {avg_memory:5.1f} GB memory, {avg_util:5.1f}% util, {avg_throughput:6.1f} samples/sec")
    
    print(f"\n💡 Most memory-efficient: Batch size {memory_efficient_idx['batch_size']}")
    print(f"   Efficiency ratio: {memory_efficient_idx['memory_efficiency']:.1f} samples/sec/GB")
    
    print("\n" + "="*60 + "\n")
    
    # 3. Training Time Analysis
    print("3. TRAINING TIME ANALYSIS:")
    print("-" * 30)
    
    # Calculate time per epoch (assuming 10 epochs)
    clean_df['time_per_epoch'] = clean_df['elapsed_time_seconds'] / 10  # 10 epochs in the experiments
    
    fastest_idx = clean_df.loc[clean_df['elapsed_time_seconds'].idxmin()]
    print(f"⏱️  Fastest training: {fastest_idx['elapsed_time_seconds']:.0f} seconds ({fastest_idx['elapsed_time_seconds']/60:.1f} minutes)")
    print(f"   Batch size: {fastest_idx['batch_size']}")
    print(f"   Time per epoch: {fastest_idx['time_per_epoch']:.1f} seconds")
    
    # Time efficiency analysis
    print(f"\n📈 Training Time by Batch Size:")
    time_summary = clean_df.groupby('batch_size')['elapsed_time_seconds'].agg(['mean', 'std']).round(1)
    for batch_size, row in time_summary.iterrows():
        minutes = row['mean'] / 60
        print(f"   Batch {batch_size:4d}: {row['mean']:6.1f}s ({minutes:4.1f} min)")
    
    print("\n" + "="*60 + "\n")
    
    # 4. Model Performance Analysis
    print("4. MODEL PERFORMANCE ANALYSIS:")
    print("-" * 35)
    
    # Loss analysis
    loss_summary = clean_df.groupby('batch_size').agg({
        'final_train_loss': ['mean', 'std'],
        'final_eval_loss': ['mean', 'std'],
        'best_eval_loss': ['mean', 'std']
    }).round(4)
    loss_summary.columns = ['_'.join(col).strip() for col in loss_summary.columns]
    
    print("📉 Loss Performance by Batch Size:")
    print(loss_summary)
    
    # Find best model performance
    best_model_idx = clean_df.loc[clean_df['best_eval_loss'].idxmin()]
    print(f"\n🎯 Best model performance:")
    print(f"   Batch size: {best_model_idx['batch_size']}")
    print(f"   Best eval loss: {best_model_idx['best_eval_loss']:.4f}")
    print(f"   Final train loss: {best_model_idx['final_train_loss']:.4f}")
    print(f"   Training throughput: {best_model_idx['training_samples_per_second']:.1f} samples/sec")
    
    return clean_df

# Perform the analysis
if len(completed_experiments) > 0:
    analysis_df = analyze_batch_size_performance(completed_experiments)
else:
    print("No completed experiments found for analysis")

=== TTM Batch Size Performance Analysis on L40S GPU ===

1. TRAINING THROUGHPUT ANALYSIS:
----------------------------------------
            training_samples_per_second_mean  training_samples_per_second_std  \
batch_size                                                                      
64                                   1641.31                              NaN   
128                                  2728.47                              NaN   
256                                  4040.03                              NaN   
512                                  5300.70                              NaN   
1024                                 5739.50                              NaN   
2048                                 6169.58                              NaN   
4096                                 6024.56                              NaN   

            elapsed_time_seconds_mean  elapsed_time_seconds_std  \
batch_size                                                        
64   

## 4. Key Recommendations

Based on the analysis above, here are our recommendations for optimal TTM training settings on the L40S GPU:

In [11]:
# Generate final recommendations
def generate_recommendations(df):
    """Generate actionable recommendations based on the analysis"""
    
    print("🎯 OPTIMAL TTM TRAINING SETTINGS FOR L40S GPU")
    print("=" * 55)
    
    # Find the sweet spot batch sizes
    clean_df = df.dropna(subset=['batch_size', 'training_samples_per_second', 'peak_memory_usage'])
    clean_df = clean_df[clean_df['batch_size'] != 'unknown']
    
    # Calculate efficiency metrics
    clean_df['speed_efficiency'] = clean_df['training_samples_per_second'] 
    clean_df['memory_efficiency'] = clean_df['training_samples_per_second'] / clean_df['peak_memory_usage']
    clean_df['overall_efficiency'] = (
        clean_df['speed_efficiency'] * 0.4 + 
        clean_df['memory_efficiency'] * 0.3 + 
        (100 - clean_df['peak_memory_usage']) * 0.2 +  # Lower memory usage is better
        clean_df['peak_gpu_utilization'] * 0.1
    )
    
    # Find optimal configurations
    best_speed_idx = clean_df.loc[clean_df['speed_efficiency'].idxmax()]
    best_memory_idx = clean_df.loc[clean_df['memory_efficiency'].idxmax()]
    best_overall_idx = clean_df.loc[clean_df['overall_efficiency'].idxmax()]
    
    print("\n1. 🚀 FOR MAXIMUM TRAINING SPEED:")
    print(f"   • Batch Size: {best_speed_idx['batch_size']}")
    print(f"   • Expected Throughput: {best_speed_idx['training_samples_per_second']:.0f} samples/sec")
    print(f"   • Memory Usage: {best_speed_idx['peak_memory_usage']:.1f} GB")
    print(f"   • GPU Utilization: {best_speed_idx['peak_gpu_utilization']:.0f}%")
    print(f"   • Training Time: ~{best_speed_idx['elapsed_time_seconds']/60:.1f} minutes per 10 epochs")
    
    print("\n2. 💡 FOR MEMORY EFFICIENCY:")
    print(f"   • Batch Size: {best_memory_idx['batch_size']}")
    print(f"   • Memory Efficiency: {best_memory_idx['memory_efficiency']:.1f} samples/sec per GB")
    print(f"   • Memory Usage: {best_memory_idx['peak_memory_usage']:.1f} GB")
    print(f"   • Throughput: {best_memory_idx['training_samples_per_second']:.0f} samples/sec")
    
    print("\n3. ⚖️ FOR BALANCED PERFORMANCE:")
    print(f"   • Batch Size: {best_overall_idx['batch_size']}")
    print(f"   • Throughput: {best_overall_idx['training_samples_per_second']:.0f} samples/sec")
    print(f"   • Memory Usage: {best_overall_idx['peak_memory_usage']:.1f} GB")
    print(f"   • GPU Utilization: {best_overall_idx['peak_gpu_utilization']:.0f}%")
    
    print("\\n" + "=" * 55)
    print("🔍 GENERAL INSIGHTS:")
    print("=" * 55)
    
    # General insights
    max_memory = clean_df['peak_memory_usage'].max()
    min_memory = clean_df['peak_memory_usage'].min()
    max_speed = clean_df['training_samples_per_second'].max()
    min_speed = clean_df['training_samples_per_second'].min()
    
    print(f"• Memory Usage Range: {min_memory:.1f} - {max_memory:.1f} GB")
    print(f"• Throughput Range: {min_speed:.0f} - {max_speed:.0f} samples/sec")
    print(f"• L40S GPU Memory: 48 GB (Peak usage: {max_memory/48*100:.1f}% of total)")
    
    # Performance scaling insights
    batch_sizes = sorted(clean_df['batch_size'].unique())
    if len(batch_sizes) >= 3:
        small_batch = clean_df[clean_df['batch_size'] == batch_sizes[0]]['training_samples_per_second'].mean()
        large_batch = clean_df[clean_df['batch_size'] == batch_sizes[-1]]['training_samples_per_second'].mean()
        speedup = large_batch / small_batch
        print(f"• Throughput scaling: {speedup:.1f}x speedup from batch {batch_sizes[0]} to {batch_sizes[-1]}")
    
    print("\n💡 PRACTICAL RECOMMENDATIONS:")
    print("-" * 30)
    print("• For rapid experimentation: Use smaller batch sizes (64-256) for faster iteration")
    print("• For production training: Use larger batch sizes (1024-4096) for maximum throughput")
    print("• Monitor GPU memory usage - the L40S has plenty of headroom for larger batches")
    print("• Consider mixed precision training to potentially double throughput")
    print("• Larger batch sizes show better GPU utilization but may require learning rate adjustments")
    
    return clean_df

# Generate recommendations
if len(completed_experiments) > 0:
    recommendations_df = generate_recommendations(completed_experiments)
else:
    print("No data available for generating recommendations")

🎯 OPTIMAL TTM TRAINING SETTINGS FOR L40S GPU

1. 🚀 FOR MAXIMUM TRAINING SPEED:
   • Batch Size: 2048
   • Expected Throughput: 6170 samples/sec
   • Memory Usage: 73.0 GB
   • GPU Utilization: 72%
   • Training Time: ~4.2 minutes per 10 epochs

2. 💡 FOR MEMORY EFFICIENCY:
   • Batch Size: 64
   • Memory Efficiency: 1641.3 samples/sec per GB
   • Memory Usage: 1.0 GB
   • Throughput: 1641 samples/sec

3. ⚖️ FOR BALANCED PERFORMANCE:
   • Batch Size: 2048
   • Throughput: 6170 samples/sec
   • Memory Usage: 73.0 GB
   • GPU Utilization: 72%
\n=======================================================
🔍 GENERAL INSIGHTS:
• Memory Usage Range: 1.0 - 98.0 GB
• Throughput Range: 1641 - 6170 samples/sec
• L40S GPU Memory: 48 GB (Peak usage: 204.2% of total)
• Throughput scaling: 3.7x speedup from batch 64 to 4096

💡 PRACTICAL RECOMMENDATIONS:
------------------------------
• For rapid experimentation: Use smaller batch sizes (64-256) for faster iteration
• For production training: Use larger bat

### 4.1. Batch Size Sweet Spot Analysis

Let's investigate whether using batch sizes between 2048 and 4096 (non-powers of 2) might offer advantages by analyzing the performance trends and interpolating between our measured data points.

In [12]:
# Analyze intermediate batch size potential
def analyze_batch_size_sweet_spot(df):
    """Analyze whether intermediate batch sizes between 2048 and 4096 might be beneficial"""
    
    print("🔍 BATCH SIZE SWEET SPOT ANALYSIS")
    print("=" * 50)
    
    # Get clean data for batch size experiments
    clean_df = df.dropna(subset=['batch_size', 'training_samples_per_second', 'peak_memory_usage'])
    clean_df = clean_df[clean_df['batch_size'] != 'unknown'].copy()
    clean_df = clean_df.sort_values('batch_size')
    
    print("Current batch size data points:")
    for _, row in clean_df.iterrows():
        print(f"   Batch {int(row['batch_size']):4d}: {row['training_samples_per_second']:6.1f} samples/sec, "
              f"{row['peak_memory_usage']:4.1f} GB memory, {row['peak_gpu_utilization']:3.0f}% GPU util")
    
    print("\n" + "="*50)
    print("📊 PERFORMANCE TREND ANALYSIS:")
    print("="*50)
    
    # Focus on the 2048 and 4096 region
    high_batch_data = clean_df[clean_df['batch_size'] >= 1024].copy()
    
    if len(high_batch_data) >= 3:
        # Calculate performance deltas
        batch_2048 = high_batch_data[high_batch_data['batch_size'] == 2048]
        batch_4096 = high_batch_data[high_batch_data['batch_size'] == 4096]
        
        if len(batch_2048) > 0 and len(batch_4096) > 0:
            throughput_2048 = batch_2048['training_samples_per_second'].iloc[0]
            throughput_4096 = batch_4096['training_samples_per_second'].iloc[0]
            memory_2048 = batch_2048['peak_memory_usage'].iloc[0]
            memory_4096 = batch_4096['peak_memory_usage'].iloc[0]
            gpu_util_2048 = batch_2048['peak_gpu_utilization'].iloc[0]
            gpu_util_4096 = batch_4096['peak_gpu_utilization'].iloc[0]
            
            throughput_gain = throughput_4096 - throughput_2048
            memory_increase = memory_4096 - memory_2048
            util_increase = gpu_util_4096 - gpu_util_2048
            
            print(f"Performance jump from 2048 → 4096:")
            print(f"   Throughput gain: +{throughput_gain:.1f} samples/sec ({throughput_gain/throughput_2048*100:.1f}%)")
            print(f"   Memory increase: +{memory_increase:.1f} GB ({memory_increase/memory_2048*100:.1f}%)")
            print(f"   GPU utilization: +{util_increase:.1f}% ({util_increase/gpu_util_2048*100:.1f}%)")
            
            # Calculate efficiency metrics
            throughput_per_gb_2048 = throughput_2048 / memory_2048
            throughput_per_gb_4096 = throughput_4096 / memory_4096
            
            print(f"\nMemory efficiency comparison:")
            print(f"   Batch 2048: {throughput_per_gb_2048:.1f} samples/sec per GB")
            print(f"   Batch 4096: {throughput_per_gb_4096:.1f} samples/sec per GB")
            print(f"   Efficiency change: {throughput_per_gb_4096 - throughput_per_gb_2048:.1f} samples/sec per GB")
            
            print("\n" + "="*50)
            print("🎯 INTERMEDIATE BATCH SIZE POTENTIAL:")
            print("="*50)
            
            # Simple linear interpolation for potential intermediate values
            potential_batch_sizes = [2560, 3072, 3584]  # Between 2048 and 4096
            
            print("Estimated performance for intermediate batch sizes:")
            print("(Linear interpolation - actual results may vary)\n")
            
            for intermediate_batch in potential_batch_sizes:
                # Linear interpolation
                ratio = (intermediate_batch - 2048) / (4096 - 2048)
                
                est_throughput = throughput_2048 + ratio * throughput_gain
                est_memory = memory_2048 + ratio * memory_increase
                est_gpu_util = gpu_util_2048 + ratio * util_increase
                est_efficiency = est_throughput / est_memory
                
                print(f"   Batch {intermediate_batch:4d}: ~{est_throughput:6.1f} samples/sec, "
                      f"~{est_memory:4.1f} GB, ~{est_gpu_util:3.0f}% util, "
                      f"{est_efficiency:.1f} eff")
            
            print("\n" + "="*50)
            print("💡 POWER-OF-2 vs NON-POWER-OF-2 CONSIDERATIONS:")
            print("="*50)
            
            print("✅ Advantages of Powers of 2 (64, 128, 256, 512, 1024, 2048, 4096):")
            print("   • Better memory alignment and cache efficiency")
            print("   • Optimal for GPU memory coalescing")
            print("   • Hardware-friendly for parallel processing")
            print("   • Most libraries are optimized for power-of-2 batch sizes")
            print("   • Easier memory management and buffer allocation")
            
            print("\n🤔 Potential Benefits of Intermediate Sizes (e.g., 3072):")
            print("   • Might hit a 'sweet spot' in GPU utilization")
            print("   • Could maximize memory usage without waste")
            print("   • May reduce overhead from underutilized GPU cores")
            print("   • Potentially better gradient stability")
            
            print("\n⚠️ Potential Drawbacks of Non-Power-of-2:")
            print("   • Less efficient memory access patterns")
            print("   • Possible performance penalties in CUDA kernels")
            print("   • May not fully utilize tensor cores")
            print("   • Suboptimal for distributed training")
            
            # Make recommendations
            print("\n" + "="*50)
            print("🎯 RECOMMENDATIONS:")
            print("="*50)
            
            if throughput_gain > 0:
                gain_per_step = throughput_gain / 2  # Rough estimate for intermediate gains
                
                print(f"Based on your L40S GPU performance data:")
                print(f"1. 📈 Performance is still improving from 2048→4096 (+{throughput_gain:.0f} samples/sec)")
                print(f"2. 🧠 Memory usage is reasonable ({memory_4096:.1f} GB / 48 GB = {memory_4096/48*100:.1f}%)")
                print(f"3. ⚡ GPU utilization increased to {gpu_util_4096:.0f}%")
                
                if memory_4096 < 40:  # Still have headroom
                    print(f"\n💡 SUGGESTED EXPERIMENTS:")
                    print(f"   • Try batch size 3072 - likely ~{throughput_2048 + gain_per_step:.0f} samples/sec")
                    print(f"   • Could be a good compromise between memory usage and throughput")
                    print(f"   • If performance is linear, you might capture most of the 4096 benefits")
                    print(f"   • Use 3072 if 4096 causes memory issues in longer training runs")
                else:
                    print(f"\n⚠️  Memory is getting tight at 4096 - stick with power-of-2 sizes")
                    
                print(f"\n🏁 PRACTICAL RECOMMENDATION:")
                if throughput_per_gb_4096 > throughput_per_gb_2048:
                    print(f"   • Stick with 4096 for maximum performance ({throughput_4096:.0f} samples/sec)")
                    print(f"   • Use 2048 for memory-constrained scenarios")
                    print(f"   • Powers of 2 are likely optimal for your setup")
                else:
                    print(f"   • Consider 2048 as the sweet spot (better efficiency)")
                    print(f"   • Test 3072 only if you need slightly higher throughput")
            else:
                print("Performance appears to plateau - stick with 2048")
                
        else:
            print("Insufficient data for 2048 and 4096 comparison")
    else:
        print("Need more high batch size data points for analysis")
    
    return clean_df

# Run the batch size sweet spot analysis
if len(completed_experiments) > 0:
    sweet_spot_df = analyze_batch_size_sweet_spot(completed_experiments)
else:
    print("No batch size experiment data available")

🔍 BATCH SIZE SWEET SPOT ANALYSIS
Current batch size data points:
   Batch   64: 1641.3 samples/sec,  1.0 GB memory,  20% GPU util
   Batch  128: 2728.5 samples/sec, 25.0 GB memory,  38% GPU util
   Batch  256: 4040.0 samples/sec, 14.0 GB memory,  38% GPU util
   Batch  512: 5300.7 samples/sec, 29.0 GB memory,  42% GPU util
   Batch 1024: 5739.5 samples/sec, 46.0 GB memory,  53% GPU util
   Batch 2048: 6169.6 samples/sec, 73.0 GB memory,  72% GPU util
   Batch 4096: 6024.6 samples/sec, 98.0 GB memory,  98% GPU util

📊 PERFORMANCE TREND ANALYSIS:
Performance jump from 2048 → 4096:
   Throughput gain: +-145.0 samples/sec (-2.4%)
   Memory increase: +25.0 GB (34.2%)
   GPU utilization: +26.0% (36.1%)

Memory efficiency comparison:
   Batch 2048: 84.5 samples/sec per GB
   Batch 4096: 61.5 samples/sec per GB
   Efficiency change: -23.0 samples/sec per GB

🎯 INTERMEDIATE BATCH SIZE POTENTIAL:
Estimated performance for intermediate batch sizes:
(Linear interpolation - actual results may vary)

In [13]:
# Create visualization showing potential intermediate batch sizes
def visualize_batch_size_interpolation(df):
    """Visualize actual data points and interpolated intermediate batch sizes"""
    
    clean_df = df.dropna(subset=['batch_size', 'training_samples_per_second', 'peak_memory_usage'])
    clean_df = clean_df[clean_df['batch_size'] != 'unknown'].copy()
    clean_df = clean_df.sort_values('batch_size')
    
    if len(clean_df) < 3:
        print("Insufficient data for interpolation visualization")
        return
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            "Throughput vs Batch Size (with Interpolation)",
            "Memory Usage vs Batch Size", 
            "GPU Utilization vs Batch Size",
            "Memory Efficiency vs Batch Size"
        ],
        vertical_spacing=0.15,
        horizontal_spacing=0.1
    )
    
    # Actual data points
    actual_batch_sizes = clean_df['batch_size'].values
    actual_throughput = clean_df['training_samples_per_second'].values
    actual_memory = clean_df['peak_memory_usage'].values
    actual_gpu_util = clean_df['peak_gpu_utilization'].values
    actual_efficiency = actual_throughput / actual_memory
    
    # Create interpolated points between 2048 and 4096
    if 2048 in actual_batch_sizes and 4096 in actual_batch_sizes:
        idx_2048 = list(actual_batch_sizes).index(2048)
        idx_4096 = list(actual_batch_sizes).index(4096)
        
        # Intermediate batch sizes
        intermediate_batches = np.array([2048, 2560, 3072, 3584, 4096])
        
        # Linear interpolation for each metric
        interp_throughput = np.interp(intermediate_batches, 
                                    [2048, 4096], 
                                    [actual_throughput[idx_2048], actual_throughput[idx_4096]])
        interp_memory = np.interp(intermediate_batches,
                                [2048, 4096],
                                [actual_memory[idx_2048], actual_memory[idx_4096]])
        interp_gpu_util = np.interp(intermediate_batches,
                                  [2048, 4096],
                                  [actual_gpu_util[idx_2048], actual_gpu_util[idx_4096]])
        interp_efficiency = interp_throughput / interp_memory
        
        # Plot 1: Throughput
        fig.add_trace(
            go.Scatter(
                x=actual_batch_sizes,
                y=actual_throughput,
                mode='markers+lines',
                name='Actual Data',
                marker=dict(size=12, color='blue'),
                line=dict(color='blue', width=3),
                hovertemplate="<b>Actual</b><br>Batch: %{x}<br>Throughput: %{y:.0f}<extra></extra>"
            ),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Scatter(
                x=intermediate_batches,
                y=interp_throughput,
                mode='markers+lines',
                name='Interpolated',
                marker=dict(size=8, color='red', symbol='diamond'),
                line=dict(color='red', width=2, dash='dash'),
                hovertemplate="<b>Interpolated</b><br>Batch: %{x}<br>Est. Throughput: %{y:.0f}<extra></extra>",
                showlegend=True
            ),
            row=1, col=1
        )
        
        # Plot 2: Memory Usage
        fig.add_trace(
            go.Scatter(
                x=actual_batch_sizes,
                y=actual_memory,
                mode='markers+lines',
                name='Actual Memory',
                marker=dict(size=12, color='green'),
                line=dict(color='green', width=3),
                showlegend=False,
                hovertemplate="<b>Actual</b><br>Batch: %{x}<br>Memory: %{y:.1f} GB<extra></extra>"
            ),
            row=1, col=2
        )
        
        fig.add_trace(
            go.Scatter(
                x=intermediate_batches,
                y=interp_memory,
                mode='markers+lines',
                name='Est. Memory',
                marker=dict(size=8, color='orange', symbol='diamond'),
                line=dict(color='orange', width=2, dash='dash'),
                showlegend=False,
                hovertemplate="<b>Interpolated</b><br>Batch: %{x}<br>Est. Memory: %{y:.1f} GB<extra></extra>"
            ),
            row=1, col=2
        )
        
        # Plot 3: GPU Utilization
        fig.add_trace(
            go.Scatter(
                x=actual_batch_sizes,
                y=actual_gpu_util,
                mode='markers+lines',
                name='Actual GPU Util',
                marker=dict(size=12, color='purple'),
                line=dict(color='purple', width=3),
                showlegend=False,
                hovertemplate="<b>Actual</b><br>Batch: %{x}<br>GPU Util: %{y:.0f}%<extra></extra>"
            ),
            row=2, col=1
        )
        
        fig.add_trace(
            go.Scatter(
                x=intermediate_batches,
                y=interp_gpu_util,
                mode='markers+lines',
                name='Est. GPU Util',
                marker=dict(size=8, color='magenta', symbol='diamond'),
                line=dict(color='magenta', width=2, dash='dash'),
                showlegend=False,
                hovertemplate="<b>Interpolated</b><br>Batch: %{x}<br>Est. GPU Util: %{y:.0f}%<extra></extra>"
            ),
            row=2, col=1
        )
        
        # Plot 4: Memory Efficiency
        fig.add_trace(
            go.Scatter(
                x=actual_batch_sizes,
                y=actual_efficiency,
                mode='markers+lines',
                name='Actual Efficiency',
                marker=dict(size=12, color='brown'),
                line=dict(color='brown', width=3),
                showlegend=False,
                hovertemplate="<b>Actual</b><br>Batch: %{x}<br>Efficiency: %{y:.1f}<extra></extra>"
            ),
            row=2, col=2
        )
        
        fig.add_trace(
            go.Scatter(
                x=intermediate_batches,
                y=interp_efficiency,
                mode='markers+lines',
                name='Est. Efficiency',
                marker=dict(size=8, color='pink', symbol='diamond'),
                line=dict(color='pink', width=2, dash='dash'),
                showlegend=False,
                hovertemplate="<b>Interpolated</b><br>Batch: %{x}<br>Est. Efficiency: %{y:.1f}<extra></extra>"
            ),
            row=2, col=2
        )
        
        # Highlight potential sweet spots
        sweet_spots = [3072]  # Example sweet spot
        for sweet_spot in sweet_spots:
            if sweet_spot in intermediate_batches:
                idx = list(intermediate_batches).index(sweet_spot)
                
                # Add vertical lines to highlight sweet spot
                for row_idx, col_idx in [(1,1), (1,2), (2,1), (2,2)]:
                    fig.add_vline(
                        x=sweet_spot, 
                        line_width=2, 
                        line_dash="dot", 
                        line_color="red",
                        annotation_text=f"Sweet Spot?",
                        annotation_position="top",
                        row=row_idx, col=col_idx
                    )
    
    fig.update_layout(
        height=800,
        width=1400,
        title_text="Batch Size Interpolation Analysis: Actual vs Estimated Performance",
        title_x=0.5,
        showlegend=True
    )
    
    # Update axis labels
    fig.update_xaxes(title_text="Batch Size", row=1, col=1)
    fig.update_xaxes(title_text="Batch Size", row=1, col=2)
    fig.update_xaxes(title_text="Batch Size", row=2, col=1)
    fig.update_xaxes(title_text="Batch Size", row=2, col=2)
    
    fig.update_yaxes(title_text="Samples/sec", row=1, col=1)
    fig.update_yaxes(title_text="Memory (GB)", row=1, col=2)
    fig.update_yaxes(title_text="GPU Util (%)", row=2, col=1)
    fig.update_yaxes(title_text="Efficiency", row=2, col=2)
    
    return fig

# Create the interpolation visualization
if len(completed_experiments) > 0:
    interp_fig = visualize_batch_size_interpolation(completed_experiments)
    if interp_fig:
        interp_fig.show()
else:
    print("No batch size data available for interpolation")

## 5. GPU Monitoring Analysis

Let's analyze the detailed GPU monitoring logs to understand resource utilization patterns during training across different batch sizes.

In [14]:
# Function to load and parse GPU monitoring logs
def load_gpu_monitoring_data(base_path, experiment_dirs):
    """Load GPU monitoring logs from batch size experiments"""
    
    gpu_data = []
    
    # Map directories to batch sizes
    batch_size_map = {
        "run_20251005_193758_job1354928": 4096,
        "run_20251005_193339_job1354927": 2048,  
        "run_20251005_192902_job1354926": 1024,
        "run_20251005_192407_job1354925": 512,
        "run_20251005_191752_job1354924": 256,
        "run_20251005_190854_job1354923": 128,
        "run_20251005_185420_job1354922": 64
    }
    
    for run_dir in experiment_dirs:
        gpu_log_path = Path(base_path) / "results" / "runs" / "ttm_finetune" / run_dir / "gpu_monitoring.log"
        
        if not gpu_log_path.exists():
            print(f"GPU log not found: {gpu_log_path}")
            continue
            
        batch_size = batch_size_map.get(run_dir, 'unknown')
        print(f"Loading GPU monitoring data for batch size {batch_size}: {gpu_log_path}")
        
        try:
            # Read the GPU monitoring log
            with open(gpu_log_path, 'r') as f:
                lines = f.readlines()
            
            # Skip header lines (starting with #)
            data_lines = [line.strip() for line in lines if not line.startswith('#') and line.strip()]
            
            for line in data_lines:
                parts = line.split()
                if len(parts) >= 23:  # Ensure we have all columns
                    try:
                        gpu_record = {
                            'batch_size': batch_size,
                            'run_dir': run_dir,
                            'date': parts[0],
                            'time': parts[1],
                            'datetime': pd.to_datetime(f"{parts[0]} {parts[1]}", format="%Y%m%d %H:%M:%S"),
                            'gpu_idx': int(parts[2]),
                            'power_w': float(parts[3]),
                            'gpu_temp_c': float(parts[4]) if parts[4] != '-' else None,
                            'mem_temp_c': float(parts[5]) if parts[5] != '-' else None,
                            'sm_util_pct': float(parts[6]),
                            'mem_util_pct': float(parts[7]),
                            'enc_util_pct': float(parts[8]),
                            'dec_util_pct': float(parts[9]),
                            'jpg_util_pct': float(parts[10]),
                            'ofa_util_pct': float(parts[11]),
                            'mem_clock_mhz': float(parts[12]),
                            'proc_clock_mhz': float(parts[13]),
                            'power_viol_pct': float(parts[14]),
                            'temp_viol': parts[15] == '1',
                            'fb_memory_mb': float(parts[16]),  # Frame buffer memory in MB
                            'bar1_mb': float(parts[17]),
                            'ccpm_mb': float(parts[18]),
                            'sbecc_errs': int(parts[19]),
                            'dbecc_errs': int(parts[20]),
                            'pci_errs': int(parts[21]),
                            'rx_pci_mbs': float(parts[22]),
                            'tx_pci_mbs': float(parts[23])
                        }
                        gpu_data.append(gpu_record)
                    except (ValueError, IndexError) as e:
                        print(f"Error parsing line in {run_dir}: {line[:50]}... - {e}")
                        continue
                        
        except Exception as e:
            print(f"Error reading {gpu_log_path}: {e}")
            continue
    
    return pd.DataFrame(gpu_data)

# Load GPU monitoring data for the batch size experiments
gpu_experiment_dirs = [
    "run_20251005_193758_job1354928",  # 4096 batch size
    "run_20251005_193339_job1354927",  # 2048 batch size
    "run_20251005_192902_job1354926",  # 1024 batch size  
    "run_20251005_192407_job1354925",  # 512 batch size
    "run_20251005_191752_job1354924",  # 256 batch size
    "run_20251005_190854_job1354923",  # 128 batch size
    "run_20251005_185420_job1354922"   # 64 batch size
]

base_path = "/u6/cjrisi/nocturnal"
gpu_df = load_gpu_monitoring_data(base_path, gpu_experiment_dirs)

print(f"Loaded {len(gpu_df)} GPU monitoring records")
print(f"Unique batch sizes: {sorted(gpu_df['batch_size'].unique())}")
print(f"Time range: {gpu_df['datetime'].min()} to {gpu_df['datetime'].max()}")

# Show sample data
gpu_df.head()

Loading GPU monitoring data for batch size 4096: /u6/cjrisi/nocturnal/results/runs/ttm_finetune/run_20251005_193758_job1354928/gpu_monitoring.log
Loading GPU monitoring data for batch size 2048: /u6/cjrisi/nocturnal/results/runs/ttm_finetune/run_20251005_193339_job1354927/gpu_monitoring.log
Loading GPU monitoring data for batch size 1024: /u6/cjrisi/nocturnal/results/runs/ttm_finetune/run_20251005_192902_job1354926/gpu_monitoring.log
Loading GPU monitoring data for batch size 512: /u6/cjrisi/nocturnal/results/runs/ttm_finetune/run_20251005_192407_job1354925/gpu_monitoring.log
Loading GPU monitoring data for batch size 256: /u6/cjrisi/nocturnal/results/runs/ttm_finetune/run_20251005_191752_job1354924/gpu_monitoring.log
Loading GPU monitoring data for batch size 128: /u6/cjrisi/nocturnal/results/runs/ttm_finetune/run_20251005_190854_job1354923/gpu_monitoring.log
Loading GPU monitoring data for batch size 64: /u6/cjrisi/nocturnal/results/runs/ttm_finetune/run_20251005_185420_job1354922/gp

,batch_size,run_dir,date,time,datetime,gpu_idx,power_w,gpu_temp_c,mem_temp_c,sm_util_pct,...,power_viol_pct,temp_viol,fb_memory_mb,bar1_mb,ccpm_mb,sbecc_errs,dbecc_errs,pci_errs,rx_pci_mbs,tx_pci_mbs
0,4096,run_20251005_193758_job1354928,20251005,19:38:04,2025-10-05 19:38:04,0,50.0,33.0,None,0.0,...,0.0,False,0.0,1.0,0.0,0,0,0,0.0,0.0
1,4096,run_20251005_193758_job1354928,20251005,19:38:14,2025-10-05 19:38:14,0,36.0,32.0,None,0.0,...,0.0,False,437.0,4.0,0.0,0,0,0,0.0,1.0
2,4096,run_20251005_193758_job1354928,20251005,19:38:24,2025-10-05 19:38:24,0,81.0,33.0,None,0.0,...,0.0,False,437.0,4.0,0.0,0,0,0,0.0,0.0
3,4096,run_20251005_193758_job1354928,20251005,19:38:34,2025-10-05 19:38:34,0,204.0,39.0,None,63.0,...,0.0,False,3597.0,4.0,0.0,0,0,0,4003.0,3.0
4,4096,run_20251005_193758_job1354928,20251005,19:38:44,2025-10-05 19:38:44,0,82.0,34.0,None,0.0,...,0.0,False,3597.0,4.0,0.0,0,0,0,0.0,0.0


In [15]:
# Create comprehensive GPU monitoring visualizations
def create_gpu_monitoring_plots(df):
    """Create time-series plots of GPU resource utilization"""
    
    # Create subplots
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=[
            "Frame Buffer Memory Usage (GB) Over Time",
            "GPU Utilization (SM %) Over Time", 
            "Power Consumption (W) Over Time",
            "GPU Temperature (°C) Over Time",
            "Memory Utilization (%) Over Time",
            "Processing Clock Speed (MHz) Over Time"
        ],
        vertical_spacing=0.08,
        horizontal_spacing=0.1
    )
    
    # Color palette for different batch sizes
    colors = px.colors.qualitative.Set1
    batch_sizes = sorted(df['batch_size'].unique())
    
    for i, batch_size in enumerate(batch_sizes):
        if batch_size == 'unknown':
            continue
            
        batch_data = df[df['batch_size'] == batch_size].copy()
        batch_data = batch_data.sort_values('datetime')
        
        # Create relative time from start of each experiment
        start_time = batch_data['datetime'].min()
        batch_data['relative_time_min'] = (batch_data['datetime'] - start_time).dt.total_seconds() / 60
        
        color = colors[i % len(colors)]
        name = f"Batch {batch_size}"
        
        # 1. Frame Buffer Memory (convert MB to GB)
        fig.add_trace(
            go.Scatter(
                x=batch_data['relative_time_min'],
                y=batch_data['fb_memory_mb'] / 1024,  # Convert MB to GB
                mode='lines',
                name=name,
                line=dict(color=color, width=2),
                showlegend=True,
                legendgroup=name,
                hovertemplate=f"<b>{name}</b><br>" +
                             "Time: %{x:.1f} min<br>" +
                             "Memory: %{y:.1f} GB<extra></extra>"
            ),
            row=1, col=1
        )
        
        # 2. GPU SM Utilization
        fig.add_trace(
            go.Scatter(
                x=batch_data['relative_time_min'],
                y=batch_data['sm_util_pct'],
                mode='lines',
                name=name,
                line=dict(color=color, width=2),
                showlegend=False,
                legendgroup=name,
                hovertemplate=f"<b>{name}</b><br>" +
                             "Time: %{x:.1f} min<br>" +
                             "SM Util: %{y:.1f}%<extra></extra>"
            ),
            row=1, col=2
        )
        
        # 3. Power Consumption
        fig.add_trace(
            go.Scatter(
                x=batch_data['relative_time_min'],
                y=batch_data['power_w'],
                mode='lines',
                name=name,
                line=dict(color=color, width=2),
                showlegend=False,
                legendgroup=name,
                hovertemplate=f"<b>{name}</b><br>" +
                             "Time: %{x:.1f} min<br>" +
                             "Power: %{y:.0f} W<extra></extra>"
            ),
            row=2, col=1
        )
        
        # 4. GPU Temperature
        temp_data = batch_data.dropna(subset=['gpu_temp_c'])
        if len(temp_data) > 0:
            fig.add_trace(
                go.Scatter(
                    x=temp_data['relative_time_min'],
                    y=temp_data['gpu_temp_c'],
                    mode='lines',
                    name=name,
                    line=dict(color=color, width=2),
                    showlegend=False,
                    legendgroup=name,
                    hovertemplate=f"<b>{name}</b><br>" +
                                 "Time: %{x:.1f} min<br>" +
                                 "Temp: %{y:.0f}°C<extra></extra>"
                ),
                row=2, col=2
            )
        
        # 5. Memory Utilization
        fig.add_trace(
            go.Scatter(
                x=batch_data['relative_time_min'],
                y=batch_data['mem_util_pct'],
                mode='lines',
                name=name,
                line=dict(color=color, width=2),
                showlegend=False,
                legendgroup=name,
                hovertemplate=f"<b>{name}</b><br>" +
                             "Time: %{x:.1f} min<br>" +
                             "Mem Util: %{y:.1f}%<extra></extra>"
            ),
            row=3, col=1
        )
        
        # 6. Processing Clock Speed
        fig.add_trace(
            go.Scatter(
                x=batch_data['relative_time_min'],
                y=batch_data['proc_clock_mhz'],
                mode='lines',
                name=name,
                line=dict(color=color, width=2),
                showlegend=False,
                legendgroup=name,
                hovertemplate=f"<b>{name}</b><br>" +
                             "Time: %{x:.1f} min<br>" +
                             "Clock: %{y:.0f} MHz<extra></extra>"
            ),
            row=3, col=2
        )
    
    # Update layout
    fig.update_layout(
        height=1200,
        width=1400,
        title_text="GPU Resource Utilization During TTM Training - Batch Size Comparison",
        title_x=0.5,
        showlegend=True
    )
    
    # Update axis labels
    for row in range(1, 4):
        for col in range(1, 3):
            fig.update_xaxes(title_text="Time (minutes)", row=row, col=col)
    
    fig.update_yaxes(title_text="Memory (GB)", row=1, col=1)
    fig.update_yaxes(title_text="Utilization (%)", row=1, col=2)
    fig.update_yaxes(title_text="Power (Watts)", row=2, col=1)
    fig.update_yaxes(title_text="Temperature (°C)", row=2, col=2)
    fig.update_yaxes(title_text="Utilization (%)", row=3, col=1)
    fig.update_yaxes(title_text="Clock Speed (MHz)", row=3, col=2)
    
    return fig

# Create GPU monitoring plots
if len(gpu_df) > 0:
    gpu_fig = create_gpu_monitoring_plots(gpu_df)
    gpu_fig.show()
else:
    print("No GPU monitoring data available for visualization")

In [16]:
# Analyze GPU utilization patterns and efficiency
def analyze_gpu_utilization(df):
    """Analyze detailed GPU utilization patterns across batch sizes"""
    
    print("🔍 DETAILED GPU UTILIZATION ANALYSIS")
    print("=" * 50)
    
    # Group by batch size and calculate statistics
    gpu_stats = df.groupby('batch_size').agg({
        'fb_memory_mb': ['mean', 'max', 'std'],
        'sm_util_pct': ['mean', 'max', 'std'],
        'mem_util_pct': ['mean', 'max', 'std'],
        'power_w': ['mean', 'max', 'std'],
        'gpu_temp_c': ['mean', 'max', 'std'],
        'proc_clock_mhz': ['mean', 'std']
    }).round(2)
    
    print("\n📊 GPU RESOURCE UTILIZATION BY BATCH SIZE:")
    print("-" * 50)
    
    for batch_size in sorted(df['batch_size'].unique()):
        if batch_size == 'unknown':
            continue
            
        batch_data = df[df['batch_size'] == batch_size]
        
        # Calculate key metrics
        avg_memory_gb = batch_data['fb_memory_mb'].mean() / 1024
        max_memory_gb = batch_data['fb_memory_mb'].max() / 1024
        avg_sm_util = batch_data['sm_util_pct'].mean()
        max_sm_util = batch_data['sm_util_pct'].max()
        avg_power = batch_data['power_w'].mean()
        max_power = batch_data['power_w'].max()
        avg_temp = batch_data['gpu_temp_c'].mean() if batch_data['gpu_temp_c'].notna().any() else 0
        
        # Training vs idle detection (SM utilization > 10% indicates training)
        training_data = batch_data[batch_data['sm_util_pct'] > 10]
        idle_data = batch_data[batch_data['sm_util_pct'] <= 10]
        
        training_time_min = len(training_data) * 10 / 60  # 10 second intervals
        total_time_min = len(batch_data) * 10 / 60

        print(f"\n🎯 BATCH SIZE {int(batch_size)}:")
        print(f"   Memory Usage: {avg_memory_gb:.1f} GB avg, {max_memory_gb:.1f} GB peak")
        print(f"   SM Utilization: {avg_sm_util:.1f}% avg, {max_sm_util:.1f}% peak")
        print(f"   Power Draw: {avg_power:.0f}W avg, {max_power:.0f}W peak")
        print(f"   Temperature: {avg_temp:.0f}°C avg")
        print(f"   Training Time: {training_time_min:.1f}/{total_time_min:.1f} min ({training_time_min/total_time_min*100:.1f}% active)")
        
        if len(training_data) > 0:
            training_avg_util = training_data['sm_util_pct'].mean()
            training_avg_power = training_data['power_w'].mean()
            training_avg_memory = training_data['fb_memory_mb'].mean() / 1024
            print(f"   During Training: {training_avg_util:.1f}% util, {training_avg_power:.0f}W, {training_avg_memory:.1f}GB")
    
    print("\n" + "=" * 50)
    print("⚡ POWER EFFICIENCY ANALYSIS:")
    print("=" * 50)
    
    # Calculate power efficiency metrics
    power_efficiency = []
    for batch_size in sorted(df['batch_size'].unique()):
        if batch_size == 'unknown':
            continue
            
        batch_data = df[df['batch_size'] == batch_size]
        training_data = batch_data[batch_data['sm_util_pct'] > 10]
        
        if len(training_data) > 0:
            avg_power_training = training_data['power_w'].mean()
            avg_util_training = training_data['sm_util_pct'].mean()
            
            # Power efficiency = Utilization per Watt
            efficiency = avg_util_training / avg_power_training if avg_power_training > 0 else 0
            power_efficiency.append({
                'batch_size': batch_size,
                'avg_power': avg_power_training,
                'avg_utilization': avg_util_training,
                'efficiency': efficiency
            })
    
    power_df = pd.DataFrame(power_efficiency)
    if len(power_df) > 0:
        power_df = power_df.sort_values('efficiency', ascending=False)
        
        print("\n🏆 Power Efficiency Ranking (Utilization % per Watt):")
        for _, row in power_df.iterrows():
            print(f"   Batch {int(row['batch_size']):4d}: {row['efficiency']:.3f} util%/W ({row['avg_utilization']:.1f}% @ {row['avg_power']:.0f}W)")
        
        best_efficiency = power_df.iloc[0]
        print(f"\n💡 Most power-efficient: Batch size {int(best_efficiency['batch_size'])}")
        print(f"   Efficiency: {best_efficiency['efficiency']:.3f} utilization%/Watt")
    
    print("\n" + "=" * 50)
    print("🌡️  THERMAL ANALYSIS:")
    print("=" * 50)
    
    # Thermal analysis
    for batch_size in sorted(df['batch_size'].unique()):
        if batch_size == 'unknown':
            continue
            
        batch_data = df[df['batch_size'] == batch_size]
        training_data = batch_data[batch_data['sm_util_pct'] > 10]
        
        if len(training_data) > 0 and training_data['gpu_temp_c'].notna().any():
            avg_temp_training = training_data['gpu_temp_c'].mean()
            max_temp_training = training_data['gpu_temp_c'].max()
            temp_range = training_data['gpu_temp_c'].max() - training_data['gpu_temp_c'].min()
            
            print(f"   Batch {int(batch_size):4d}: {avg_temp_training:.0f}°C avg, {max_temp_training:.0f}°C peak, {temp_range:.0f}°C range")
    
    print("\n💡 THERMAL INSIGHTS:")
    print("   • All temperatures well within safe limits (<83°C typically)")
    print("   • Higher batch sizes may run slightly warmer due to increased utilization")
    print("   • L40S GPU has excellent thermal management")
    
    return power_df if len(power_efficiency) > 0 else None

# Perform GPU utilization analysis
if len(gpu_df) > 0:
    efficiency_df = analyze_gpu_utilization(gpu_df)
else:
    print("No GPU monitoring data available for analysis")

🔍 DETAILED GPU UTILIZATION ANALYSIS



📊 GPU RESOURCE UTILIZATION BY BATCH SIZE:
--------------------------------------------------

🎯 BATCH SIZE 64:
   Memory Usage: 0.8 GB avg, 19.3 GB peak
   SM Utilization: 5.0% avg, 17.0% peak
   Power Draw: 89W avg, 107W peak
   Temperature: 33°C avg
   Training Time: 5.3/14.5 min (36.8% active)
   During Training: 12.8% util, 100W, 0.6GB

🎯 BATCH SIZE 128:
   Memory Usage: 0.6 GB avg, 0.7 GB peak
   SM Utilization: 7.2% avg, 25.0% peak
   Power Draw: 92W avg, 121W peak
   Temperature: 34°C avg
   Training Time: 3.5/8.8 min (39.6% active)
   During Training: 17.7% util, 110W, 0.7GB

🎯 BATCH SIZE 256:
   Memory Usage: 0.7 GB avg, 0.8 GB peak
   SM Utilization: 6.0% avg, 36.0% peak
   Power Draw: 91W avg, 152W peak
   Temperature: 34°C avg
   Training Time: 1.5/6.2 min (24.3% active)
   During Training: 24.6% util, 126W, 0.8GB

🎯 BATCH SIZE 512:
   Memory Usage: 0.9 GB avg, 1.0 GB peak
   SM Utilization: 6.3% avg, 39.0% peak
   Power Draw: 102W avg, 199W peak
   Temperature: 34°C avg
 

In [17]:
# Create memory usage and efficiency analysis
def create_memory_analysis_plots(df):
    """Create detailed memory usage analysis plots"""
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            "Memory Usage Distribution by Batch Size",
            "Memory vs SM Utilization Correlation",
            "Power Consumption vs Memory Usage",
            "Memory Efficiency Timeline"
        ],
        vertical_spacing=0.12,
        horizontal_spacing=0.1
    )
    
    colors = px.colors.qualitative.Set1
    batch_sizes = sorted(df['batch_size'].unique())
    
    # 1. Memory Usage Distribution (Box Plot)
    for i, batch_size in enumerate(batch_sizes):
        if batch_size == 'unknown':
            continue
            
        batch_data = df[df['batch_size'] == batch_size]
        memory_gb = batch_data['fb_memory_mb'] / 1024
        
        fig.add_trace(
            go.Box(
                y=memory_gb,
                name=f"Batch {batch_size}",
                marker_color=colors[i % len(colors)],
                showlegend=False
            ),
            row=1, col=1
        )
    
    # 2. Memory vs SM Utilization Scatter
    for i, batch_size in enumerate(batch_sizes):
        if batch_size == 'unknown':
            continue
            
        batch_data = df[df['batch_size'] == batch_size]
        
        fig.add_trace(
            go.Scatter(
                x=batch_data['fb_memory_mb'] / 1024,
                y=batch_data['sm_util_pct'],
                mode='markers',
                name=f"Batch {batch_size}",
                marker=dict(color=colors[i % len(colors)], size=6, opacity=0.6),
                showlegend=True,
                hovertemplate=f"<b>Batch {batch_size}</b><br>" +
                             "Memory: %{x:.1f} GB<br>" +
                             "SM Util: %{y:.1f}%<extra></extra>"
            ),
            row=1, col=2
        )
    
    # 3. Power vs Memory Scatter
    for i, batch_size in enumerate(batch_sizes):
        if batch_size == 'unknown':
            continue
            
        batch_data = df[df['batch_size'] == batch_size]
        
        fig.add_trace(
            go.Scatter(
                x=batch_data['fb_memory_mb'] / 1024,
                y=batch_data['power_w'],
                mode='markers',
                name=f"Batch {batch_size}",
                marker=dict(color=colors[i % len(colors)], size=6, opacity=0.6),
                showlegend=False,
                hovertemplate=f"<b>Batch {batch_size}</b><br>" +
                             "Memory: %{x:.1f} GB<br>" +
                             "Power: %{y:.0f} W<extra></extra>"
            ),
            row=2, col=1
        )
    
    # 4. Memory Efficiency Timeline
    for i, batch_size in enumerate(batch_sizes):
        if batch_size == 'unknown':
            continue
            
        batch_data = df[df['batch_size'] == batch_size].copy()
        batch_data = batch_data.sort_values('datetime')
        
        # Calculate relative time and memory efficiency
        start_time = batch_data['datetime'].min()
        batch_data['relative_time_min'] = (batch_data['datetime'] - start_time).dt.total_seconds() / 60
        
        # Memory efficiency = SM utilization per GB of memory used
        batch_data['memory_efficiency'] = batch_data['sm_util_pct'] / (batch_data['fb_memory_mb'] / 1024)
        batch_data['memory_efficiency'] = batch_data['memory_efficiency'].replace([np.inf, -np.inf], 0)
        
        fig.add_trace(
            go.Scatter(
                x=batch_data['relative_time_min'],
                y=batch_data['memory_efficiency'],
                mode='lines',
                name=f"Batch {batch_size}",
                line=dict(color=colors[i % len(colors)], width=2),
                showlegend=False,
                hovertemplate=f"<b>Batch {batch_size}</b><br>" +
                             "Time: %{x:.1f} min<br>" +
                             "Efficiency: %{y:.1f} util%/GB<extra></extra>"
            ),
            row=2, col=2
        )
    
    # Update layout
    fig.update_layout(
        height=800,
        width=1400,
        title_text="GPU Memory Usage and Efficiency Analysis",
        title_x=0.5,
        showlegend=True
    )
    
    # Update axis labels
    fig.update_xaxes(title_text="Batch Size", row=1, col=1)
    fig.update_xaxes(title_text="Memory Usage (GB)", row=1, col=2)
    fig.update_xaxes(title_text="Memory Usage (GB)", row=2, col=1)
    fig.update_xaxes(title_text="Time (minutes)", row=2, col=2)
    
    fig.update_yaxes(title_text="Memory Usage (GB)", row=1, col=1)
    fig.update_yaxes(title_text="SM Utilization (%)", row=1, col=2)
    fig.update_yaxes(title_text="Power (Watts)", row=2, col=1)
    fig.update_yaxes(title_text="Efficiency (util%/GB)", row=2, col=2)
    
    return fig

# Create memory analysis plots
if len(gpu_df) > 0:
    memory_fig = create_memory_analysis_plots(gpu_df)
    memory_fig.show()
    
    # Summary statistics
    print("\\n" + "="*60)
    print("📈 MEMORY USAGE SUMMARY STATISTICS")
    print("="*60)
    
    memory_summary = gpu_df.groupby('batch_size').agg({
        'fb_memory_mb': ['mean', 'max', 'min', 'std']
    }).round(1)
    
    memory_summary.columns = ['Mean_MB', 'Max_MB', 'Min_MB', 'Std_MB']
    memory_summary['Mean_GB'] = (memory_summary['Mean_MB'] / 1024).round(2)
    memory_summary['Max_GB'] = (memory_summary['Max_MB'] / 1024).round(2)
    memory_summary['Memory_Range_GB'] = ((memory_summary['Max_MB'] - memory_summary['Min_MB']) / 1024).round(2)
    
    print(memory_summary[['Mean_GB', 'Max_GB', 'Memory_Range_GB']])
    
else:
    print("No GPU monitoring data available for memory analysis")

\n============================================================
📈 MEMORY USAGE SUMMARY STATISTICS
            Mean_GB  Max_GB  Memory_Range_GB
batch_size                                  
64             0.80   19.32            19.32
128            0.64    0.67             0.67
256            0.72    0.77             0.77
512            0.89    0.96             0.96
1024           1.24    1.38             1.38
2048           1.90    2.13             2.13
4096           3.14    3.51             3.51
